# Mid-Sagittal plane algorithm

### Import relevant packages

In [1071]:
import joblib
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import center_of_mass
from scipy.optimize import minimize
import pandas as pd
from scipy.optimize import curve_fit
from scipy.interpolate import RegularGridInterpolator
from joblib import Parallel, delayed, dump, load
from matplotlib.widgets import Slider
import ipywidgets as widgets
from matplotlib.lines import Line2D
import time
from scipy import ndimage
import json
import skimage
from scipy.spatial import ConvexHull, distance
import seaborn as sns
import pickle
import os
import gzip
from scipy.spatial.transform import Rotation as R

## Data Reading

In [1072]:
def load_nifti_file(file_path):
    """
    Load a NIfTI file and return the image data.
    
    Parameters:
    file_path (str): Path to the NIfTI file.
    
    Returns:
    numpy.ndarray: The image data.
    """
    import nibabel as nib

    nifti_img = nib.load(file_path)
    img_data = nifti_img.get_fdata()
    voxel_size = nifti_img.header.get_zooms()
    
    return img_data, voxel_size

def open_gzip_file(gzip_file_path):
    """
    Open a gzip file and return its content.

    Parameters:
    gzip_file_path (str): The path to the gzip file.

    Returns:
    bytes: The content of the gzip file.
    """
    try:
        with gzip.open(gzip_file_path, 'rb') as f_in:
            file_content = f_in.read()
        return file_content
    except Exception as e:
        print(f'Error opening {gzip_file_path}: {e}')
        return None
    
def get_image_and_voxel_size_from_gzip(gzip_file_path):
    """
    Get the image array and voxel size from a gzipped NIfTI file.

    Parameters:
    gzip_file_path (str): Path to the gzipped NIfTI file.

    Returns:
    tuple: The image array and voxel size.
    """

    file_content = open_gzip_file(gzip_file_path)
    if file_content is not None:
        with open('temp_nifti.nii', 'wb') as temp_file:
            temp_file.write(file_content)

        # 🔹 Debugging: Check if the file was actually written
        file_size = os.path.getsize('temp_nifti.nii')
        if file_size == 0:
            print(f"❌ Error: 'temp_nifti.nii' was written but is empty! ({gzip_file_path})")
            return None, None

        img_data, voxel_size = load_nifti_file('temp_nifti.nii')
        os.remove('temp_nifti.nii')  # Remove temp file after reading
        
        return img_data, voxel_size
    else:
        print(f"❌ Error: Failed to read file content from '{gzip_file_path}'")
        return None, None
    
def load_patient_structures(patient_folder: str, structure_names=None):
    """
    Searches the patient folder (recursively) for NIfTI files matching the
    specified structure names, loads each file, transposes the image array,
    and returns a mapping from structure name to (image_array, voxel_size).

    Parameters
    ----------
    patient_folder : str
        Path to the patient folder (numeric name).
    structure_names : list of str, optional
        List of expected structure names (without extension). Defaults to:
        ['Image', 'GTVp', 'Body', 'Mandible', 'Spinal Cord'].
    Returns
    -------
    dict
        Keys are structure names from structure_names that were found,
        values are tuples (img_array, voxel_size).
    """
    if structure_names is None:
        structure_names = ['Image', 'GTVp', 'Body', 'Mandible', 'Spinal Cord']

    struct_dict = {}
    for root, _, files in os.walk(patient_folder):
        for f in files:
            # Only NIfTI files
            if not (f.endswith('.nii.gz') or f.endswith('.nii')):
                continue

            name_part = f
            
            # Entferne Extension
            if name_part.endswith('.nii.gz'):
                base = name_part[:-7]
            else:
                base = name_part[:-4]

            # Vergleiche mit erwarteten Struktur-Namen (case-insensitive)
            for struct_name in structure_names:
                if base.lower() == struct_name.lower():
                    file_path = os.path.join(root, f)
                    try:
                        if f.endswith('.nii.gz'):
                            img, voxel_size = get_image_and_voxel_size_from_gzip(file_path)
                        else:
                            img, voxel_size = load_nifti_file(file_path)
                    except Exception as e:
                        logging.error(f"Error loading '{struct_name}' from {file_path}: {e}")
                        continue

                    if img is not None:
                        #img = np.transpose(img, (1, 0, 2))
                        struct_dict[struct_name] = (img, voxel_size)
                        print(f"Loaded '{struct_name}' from {file_path}")
                    break
    return struct_dict

## Image processing

In [1073]:
def mask_via_threshold(ct_image, HU_range=(300, 1500)):
    """
    Generate a bone mask from a CT image by thresholding within a specified HU range.

    Parameters:
    ct_image (numpy.ndarray): The 3D CT image data.
    HU_range (tuple): The range of Hounsfield Units to identify bone. Default is (700, 2000).

    Returns:
    numpy.ndarray: The 3D bone mask with bone as 1 and all other as 0.
    """
    bone_mask = np.zeros_like(ct_image)
    lower_bound, upper_bound = HU_range
    bone_mask[(ct_image >= lower_bound) & (ct_image <= upper_bound)] = 1

    return bone_mask

def get_nonzero_slice_range(image_data, slice_dir_indx=2):
    """
    Get the range of slices that contain non-zero values in a 3D image.
    
    This function computes which slices along the specified dimension contain 
    any non-zero elements by collapsing the other two dimensions.
    
    Parameters
    ----------
    image_data : numpy.ndarray
        The 3D image data.
    slice_dir_indx : int, optional
        The index of the slice direction (0, 1, or 2). Default is 2.
    
    Returns
    -------
    tuple
        A tuple (start, end) where start is the first slice index and end is the last 
        slice index that contain non-zero values.
    
    Raises
    ------
    ValueError
        If no non-zero slices are found or if slice_dir_indx is not 0, 1, or 2.
    """
    if image_data.ndim != 3:
        raise ValueError("image_data must be a 3D array.")
    if slice_dir_indx not in (0, 1, 2):
        raise ValueError("slice_dir_indx must be 0, 1, or 2.")
    
    # Determine which axes to collapse for the nonzero test.
    if slice_dir_indx == 0:
        collapsed = np.any(image_data, axis=(1, 2))
    elif slice_dir_indx == 1:
        collapsed = np.any(image_data, axis=(0, 2))
    else:  # slice_dir_indx == 2
        collapsed = np.any(image_data, axis=(0, 1))
    
    nonzero_indices = np.nonzero(collapsed)[0]
    if nonzero_indices.size == 0:
        raise ValueError("No non-zero slices found in the specified direction.")
    
    start = int(nonzero_indices[0])
    end = int(nonzero_indices[-1])
    
    return start, end

## Parametrization

In [1074]:
def vector_to_angles(vector):
    
    x, y, z = vector

    # Calculate the distance from the origin
    R = np.linalg.norm(vector)

    # Calculate the distance from the origin in xy plane
    r = np.sqrt(x**2 + y**2)

    # Calculate the angle in the xy plane with respect to the x-axis
    azimuthal = np.arctan2(y, x)

    # Calculate the angle in the xz plane with respect to the z-axis
    polar = np.arctan2(r, z)

    return np.array([azimuthal, polar, R])

def angles_to_vector(azimuthal, polar, R):
    
    x = R * np.sin(polar) * np.cos(azimuthal)
    y = R * np.sin(polar) * np.sin(azimuthal)
    z = R * np.cos(polar)
    
    return np.array([x, y, z])

def generate_normal(theta: float, phi: float) -> np.ndarray:
    return np.array([
        np.sin(phi) * np.cos(theta),
        np.sin(phi) * np.sin(theta),
        np.cos(phi)
    ])

## Objective function


In [1075]:
def compute_signed_distances(params_array, image):
    
    azimuthal, polar, L = params_array

    indices_image = np.array(np.nonzero(image)).T

    indices_coord_syst = np.stack([
        indices_image[:,1],
        indices_image[:,0],
        indices_image[:,2]
    ], axis=1)

    # Unit normal in mathematical spherical convention
    n = np.array([
        np.sin(polar) * np.cos(azimuthal),
        np.sin(polar) * np.sin(azimuthal),
        np.cos(polar)
    ])

    d = indices_coord_syst.dot(n) + L

    return d, n, indices_coord_syst, indices_image

def huber_loss(diff, delta=1200):
    
    
    h_loss = np.where(np.abs(diff) <= delta, 0.5 * diff**2,
                    delta * (np.abs(diff) - 0.5 * delta))
    return h_loss

def compute_objective(params_array, image, interpolator_intensity):
    
    d, n, indices_coord_syst, indices_image = compute_signed_distances(params_array, image)
    N = len(d)
    x_m_coord_syst = indices_coord_syst - 2 * d[:, None] * n[None, :]
    x_m_image = np.array([x_m_coord_syst[:, 1], x_m_coord_syst[:, 0], x_m_coord_syst[:, 2]]).T
    I_m = interpolator_intensity(x_m_image)
    I_orig = image[indices_image[:, 0], indices_image[:, 1], indices_image[:, 2]]
    # plot original image and mirror image

    def plot_mirror_comparison(I_orig, I_m, indices_image, x_m_image, image_shape, z_slice=None):
        """
        Plots the original and mirrored intensities on a specified z-slice.

        Parameters
        ----------
        I_orig : np.ndarray, shape (N,)
            Original intensities from compute_objective.
        I_m : np.ndarray, shape (N,)
            Mirrored intensities from compute_objective.
        indices_image : np.ndarray, shape (N, 3)
            Coordinates (y, x, z) of the original voxels.
        x_m_image : np.ndarray, shape (N, 3)
            Coordinates (y, x, z) of the mirrored voxels.
        image_shape : tuple of ints (H, W, D)
            Shape of the full image volume.
        z_slice : int or None
            Index of the z-slice to plot. If None, uses the central slice.
        """
        H, W, D = image_shape
        if z_slice is None:
            z_slice = D // 2
        # ensure indices are integer
        indices_image = np.round(indices_image).astype(int)
        x_m_image     = np.round(x_m_image).astype(int)
        # Prepare empty slices
        slice_orig = np.zeros((H, W), dtype=I_orig.dtype)
        slice_mir  = np.zeros((H, W), dtype=I_orig.dtype)

        # Fill in original intensities for this z-slice
        mask_orig = indices_image[:, 2] == z_slice
        y_orig = indices_image[mask_orig, 0]
        x_orig = indices_image[mask_orig, 1]
        slice_orig[y_orig, x_orig] = I_orig[mask_orig]

        # Fill in mirrored intensities for this z-slice
        mask_mir = x_m_image[:, 2] == z_slice
        y_mir = x_m_image[mask_mir, 0]
        x_mir = x_m_image[mask_mir, 1]
        slice_mir[y_mir, x_mir] = I_m[mask_mir]

        # Plot side by side
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(slice_orig, cmap='gray', origin='lower')
        plt.title(f'Original Intensities (z={z_slice})')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(slice_mir, cmap='gray', origin='lower')
        plt.title(f'Mirrored Intensities (z={z_slice})')
        plt.axis('off')

        plt.tight_layout()
        plt.show()

    def mirror_and_sum_image(image: np.ndarray, theta: float, phi: float, L: float) -> np.ndarray:
        """
        Mirrors a 3D image across the plane defined by (theta, phi, L), then sums the original
        and mirrored images.

        Parameters
        ----------
        image : np.ndarray
            Input 3D image array of shape (H, W, D).
        theta : float
            Azimuthal angle in radians (xy-plane from +x-axis).
        phi : float
            Polar (inclination) angle in radians (from +z-axis).
        L : float
            Plane offset from the origin along the normal.

        Returns
        -------
        np.ndarray
            The sum of the original and mirrored image, same shape as `image`.
        """
        H, W, D = image.shape

        # Generate voxel indices (y, x, z) and flatten
        coords = np.indices((H, W, D))
        idx_flat = coords.reshape(3, -1).T  # shape (N, 3)

        # Convert to computation coordinates (x, y, z)
        comp_coords = np.stack([idx_flat[:,1], idx_flat[:,0], idx_flat[:,2]], axis=1)

        # Compute normal vector from angles
        n = np.array([
            np.sin(phi) * np.cos(theta),
            np.sin(phi) * np.sin(theta),
            np.cos(phi)
        ])

        # Signed distance of each point from the plane
        d = comp_coords.dot(n) - L

        # Mirror each coordinate across the plane: x_m = x - 2 * d * n
        mirrored = comp_coords - 2 * d[:, None] * n[None, :]

        # Convert mirrored coords back to image indexing (y, x, z)
        yi = np.round(mirrored[:,1]).astype(int)
        xi = np.round(mirrored[:,0]).astype(int)
        zi = np.round(mirrored[:,2]).astype(int)

        # Clip to valid ranges
        yi = np.clip(yi, 0, H-1)
        xi = np.clip(xi, 0, W-1)
        zi = np.clip(zi, 0, D-1)

        # Fetch intensities
        I_orig = image[idx_flat[:,0], idx_flat[:,1], idx_flat[:,2]]
        I_mir  = image[yi, xi, zi]

        # Sum and reshape
        sum_flat  = I_orig + I_mir
        sum_image = sum_flat.reshape(H, W, D)

        return sum_image

    
    # sum_img = mirror_and_sum_image(image, params_array[0], params_array[1], params_array[2])
    # slice_z = sum_img[:, :, sum_img.shape[2] // 2]

    # plt.imshow(slice_z, cmap='gray', origin='lower')
    # plt.title("Sum of Original and Mirrored Image (central Z-slice)")
    # plt.xlabel("X")
    # plt.ylabel("Y")
    # plt.colorbar(label="Intensity")
    # plt.show()

    #plot_mirror_comparison(I_orig, I_m, indices_image, x_m_image, image.shape)
  

    diff = I_orig - I_m
    f = (1.0 / N) * np.sum(huber_loss(diff, delta=5000))
    return f


## Parameter Initialization

In [ ]:
def parameter_initialization(image, bone, output_path, interpolator_intensity,
                         azimuthal_deg=0, polar_deg=0, uncertainty = 90,
                         initialization_steps=10):
    
    print("Starting parameter initialization...")
    start = time.time()
    com = center_of_mass(image)
    com = np.array([com[1], com[0], com[2]])
    
    # Define angular search ranges (in radians)
    azimuthal_rad = np.deg2rad(azimuthal_deg)
    polar_rad = np.deg2rad(polar_deg)
    delta = np.deg2rad(uncertainty)
    azimuthal_angles = np.linspace(azimuthal_rad - delta, azimuthal_rad + delta, initialization_steps)
    polar_angles = np.linspace(polar_rad - delta, polar_rad + delta, initialization_steps)
    print(f"Azimuthal angles: {np.rad2deg(azimuthal_angles)}")
    print(f"Polar angles: {np.rad2deg(polar_angles)}")
    
    os.makedirs(output_path, exist_ok=True)
    mse_array_file = os.path.join(output_path, f"Initialization_obj_fun.npy")
    plane_params_file = os.path.join(output_path, f"Initialization_plane_params.npy")
    
    if not (os.path.exists(mse_array_file) and os.path.exists(plane_params_file)):
        φ, θ = np.meshgrid(polar_angles, azimuthal_angles, indexing='ij')
        n = generate_normal(θ, φ)
        D = -np.tensordot(n, com, axes=([0],[0]))
        planes = np.stack([θ.ravel(), φ.ravel(), D.ravel()], axis=1)
        mse_array = np.array([compute_objective(p, bone, interpolator_intensity) for p in planes])
        # for parameter in planes:
        #     plot_middle_slice_with_planes(image, [parameter], com = com, title = f"parameters {np.rad2deg(parameter[0]):.2f} {np.rad2deg(parameter[1]):.2f} {parameter[2]:.2f}")
        np.save(mse_array_file, mse_array)
        np.save(plane_params_file, planes)
    else:
        mse_array = np.load(mse_array_file)
        planes    = np.load(plane_params_file)
    best_mse_index = np.argmin(mse_array)
    best_params = planes[best_mse_index]
    end = time.time()
    print(f"Time taken for initialization: {end - start:.2f} seconds")
    return best_params, mse_array[best_mse_index]


## Optimization

In [1077]:
def optimize_plane(initial_params_array, image, interpolator_intensity, interpolators_gradient):
    """
    Optimize the plane parameters [theta, phi, L] using the BFGS method.

    The objective function is:
      f(theta, phi, L) = (1/N) * sum_i [ I(x_i) - I(x_m,i) ]^2,
    where x_m is computed as:
      x_m = x - 2 * alpha * n,
    with
      n = [cos(phi)*cos(theta), cos(phi)*sin(theta), sin(phi)]
      alpha = cos(phi)*cos(theta)*x + cos(phi)*sin(theta)*y + sin(phi)*z - L.
    The voxel coordinates x are taken as the nonzero indices of the image.

    Parameters
    ----------
    initial_params_array : array-like, shape (3,)
        Initial guess for the parameters [theta, phi, L].
    image : 3D numpy array
        The intensity image.
    interpolator_intensity : RegularGridInterpolator
        Interpolator to evaluate the intensity at any (x, y, z) position.
    interpolators_gradient : dict
        Dictionary with keys 'x', 'y', and 'z' containing RegularGridInterpolator
        objects for the gradient components of the image.

    Returns
    -------
    res : OptimizeResult
        The optimization result returned by scipy.optimize.minimize.
        The lists `res.objective_value_list` and `res.params_list` contain the
        objective function values and parameter vectors encountered during the optimization.
    """

    # Lists to store the objective values and parameters at each iteration.
    objective_value_list = []
    params_list = []

    # Callback function to save current parameter vector and objective value.
    def callback(xk):
        # Compute current objective value using our objective function.
        f_val = compute_objective(xk, image, interpolator_intensity, interpolators_gradient)
        objective_value_list.append(f_val)
        params_list.append(xk.copy())

    
    # Run the optimizer using BFGS.
    res = minimize(compute_objective, x0=initial_params_array, args=(image, interpolator_intensity, interpolators_gradient),
                   method='BFGS', jac=None, callback=callback)
    params_list.append(res.x.copy())
    objective_value_list.append(res.fun)

    # Attach the objective and parameter histories to the result.
    res.objective_value_list = objective_value_list
    res.params_list = params_list

    return res

## Verification Plots

In [1078]:
def plot_middle_slice_with_planes(image_data, plane_params_list, title='Middle Slice with Plane Projections', com=None, output_path=None, filename="middle_slice_with_planes.svg"):
    """
    Plot the middle axial slice of a 3D image with projections of multiple planes.

    Parameters
    ----------
    image_data : numpy.ndarray
        The 3D image data.
    plane_params_list : list
        A list of plane parameters, where each element is a tuple or array of [theta, phi, L].
    title : str, optional
        The title of the plot.
    com : array-like, optional
        The center of mass (as [x, y]) to be plotted.
    output_path : str, optional
        Directory in which to save the figure. If None, the figure is not saved.
    filename : str, optional
        Filename for the saved figure.
    """
    # Compute the index for the middle axial slice.
    slice_index = int(np.round(com[2])) if com is not None else image_data.shape[2] // 2
    slice_array = image_data[:, :, slice_index]
    
    # Create the figure and axis.
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(slice_array, cmap='gray')
    ax.set_title(title)
    ax.set_xlabel("x (pixels)")
    ax.set_ylabel("y (pixels)")
    
    # Plot the center of mass if provided.
    if com is not None:
        ax.scatter(com[0], com[1], color='blue', marker='x', s=100, label='Center of Mass')
    
    # Create a grid for contour plotting.
    ny, nx = slice_array.shape
    x = np.linspace(-nx, 2*nx, 400)
    y = np.linspace(-ny, 2*ny, 400)
    X, Y = np.meshgrid(x, y)
    
    # For each plane, compute its intersection with the middle slice.
    for plane_coeffs in plane_params_list:
        theta, phi, L = plane_coeffs[0], plane_coeffs[1], plane_coeffs[2]
        vec = angles_to_vector(theta, phi, L)
        A, B, C = vec[0], vec[1], vec[2]
        D = - np.dot([A, B, C], [A, B, C])
        # Normalize the plane coefficients (make a copy so as not to modify the input)

        if np.abs(C) < 1e-8:
            C = 1e-8
        # Solve for z in the plane equation: z = (-A*x - B*y - D) / C.
        # Find the contour where z equals the middle slice index.
        contour = (-A * X - B * Y - D) / C
        ax.contour(X, Y, contour, levels=[slice_index], colors='red')
    
    if com is not None:
        ax.legend()

    # Save the figure if an output path is provided.
    if output_path is not None:
        save_path = os.path.join(output_path, filename)
        plt.savefig(save_path)
        print(f"Figure saved to {save_path}")
    
    plt.show()

## Scrollable Widget

In [1079]:
def sample_random_points(coords, distances, n_points):
    """
    Randomly sample a subset of points from the coordinate array and their corresponding distances.
    
    Parameters
    ----------
    coords : numpy.ndarray
        A 2D array of shape (N, 3) (or any shape where the first dimension is the number of points)
        containing the positions of the voxels.
    distances : numpy.ndarray
        A 1D array of length N containing the corresponding signed distances.
    n_points : int
        The number of random points to sample.
        
    Returns
    -------
    sampled_coords : numpy.ndarray
        A 2D array containing the randomly sampled coordinates.
    sampled_distances : numpy.ndarray
        A 1D array containing the distances corresponding to the sampled coordinates.
    """
    if n_points > len(distances):
        n_points = len(distances)
    # Randomly select n_points indices from the available points.
    indices = np.random.choice(len(distances), size=n_points, replace=False)
    sampled_coords = coords[indices]
    sampled_distances = distances[indices]
    return sampled_coords, sampled_distances

def display_scrollable_slices_with_plane(image, gtv_mask, body_mask=None, mandible_mask=None, spinal_cord_mask=None,
                              plane_coeffs_list=[], optimization_methods_list=[], points = None, distances = None):
    """
    Display an interactive widget to scroll through slices of the 3D image (in two views) with
    overlays of masks and plane contours.
    
    Parameters
    ----------
    image : numpy.ndarray
        The 3D image data.
    gtv_mask : numpy.ndarray
        The 3D primary GTV mask (mandatory).
    body_mask : numpy.ndarray, optional
        The 3D body mask.
    mandible_mask : numpy.ndarray, optional
        The 3D mandible mask.
    spinal_cord_mask : numpy.ndarray, optional
        The 3D spinal cord mask.
    plane_coeffs_list : list
        List of plane coefficients, where each is a tuple/list (theta, phi, L).
    optimization_methods_list : list
        List of method names corresponding to each plane, used for the legend.
    """
    num_slices = image.shape[2]
    plane_colors = ['red', 'purple', 'cyan']
    
    # Calculate center of mass from body_mask if provided.
    if body_mask is not None:
        com = center_of_mass(body_mask)
        # Swap first two coordinates to match image orientation.
        com = (com[1], com[0], com[2])
    else:
        com = None
    #points, distances = sample_random_points(points, distances, 15)

    def view_slice_axial(slice_index):
        fig, ax = plt.subplots(figsize=(10, 10))
        # Display the grayscale image.
        ax.imshow(image[:, :, slice_index], cmap='gray', interpolation='none')
        
        if points is not None:
            mask = points[:, 2] == slice_index
            filtered_coords = points[mask]
            filtered_distances = distances[mask]
            for (x, y, z), dist in zip(filtered_coords, filtered_distances):
                plt.plot(x, y, 'ro', markersize=2)
                plt.text(x, y, f"{round(dist,1)}", color='green', fontsize=6)
                
        # Overlay the mandatory GTV mask.
        ax.contour(gtv_mask[:, :, slice_index], colors='yellow', linewidths=1)
        
        # Overlay optional masks if available.
        if body_mask is not None:
            ax.contour(body_mask[:, :, slice_index], colors='orange', linewidths=1)
        if mandible_mask is not None:
            ax.contour(mandible_mask[:, :, slice_index], colors='blue', linewidths=1)
        if spinal_cord_mask is not None:
            ax.contour(spinal_cord_mask[:, :, slice_index], colors='green', linewidths=1)
            
        # Overlay each plane contour.
        for idx, coeffs in enumerate(plane_coeffs_list):
            A, B, C = angles_to_vector(coeffs[0], coeffs[1], coeffs[2])
            D = - np.dot([A, B, C], [A, B, C])
            # Avoid division by zero.
            C_val = C if C != 0 else 1e-6
            x = np.linspace(0, image.shape[1], 100)
            y = np.linspace(0, image.shape[0], 100)
            X, Y = np.meshgrid(x, y)
            contour = (-A * X - B * Y - D) / C_val
            ax.contour(X, Y, contour, levels=[slice_index],
                       colors=plane_colors[idx % len(plane_colors)], linewidths=1)
        
        # Build legend based on available overlays.
        legend_handles = [Line2D([0], [0], color='yellow', lw=2, label='GTVp')]
        if body_mask is not None:
            legend_handles.append(Line2D([0], [0], color='orange', lw=2, label='Body'))
        if mandible_mask is not None:
            legend_handles.append(Line2D([0], [0], color='blue', lw=2, label='Mandible'))
        if spinal_cord_mask is not None:
            legend_handles.append(Line2D([0], [0], color='green', lw=2, label='Spinal Cord'))
        for idx, coeffs in enumerate(plane_coeffs_list):
            if idx < len(optimization_methods_list):
                legend_handles.append(Line2D([0], [0], color=plane_colors[idx % len(plane_colors)],
                                              lw=2, label=optimization_methods_list[idx]))
        
        ax.legend(handles=legend_handles, loc='upper right')
        
        # Custom coordinate formatter for axial view.
        def format_coord(x, y):
            col = int(round(x))
            row = int(round(y))
            info = f"x={col}, y={row}, slice={slice_index}"
            if 0 <= row < gtv_mask.shape[0] and 0 <= col < gtv_mask.shape[1]:
                info += f", GTVp={gtv_mask[row, col, slice_index]}"
            if body_mask is not None and 0 <= row < body_mask.shape[0] and 0 <= col < body_mask.shape[1]:
                info += f", Body={body_mask[row, col, slice_index]}"
            if mandible_mask is not None and 0 <= row < mandible_mask.shape[0] and 0 <= col < mandible_mask.shape[1]:
                info += f", Mandible={mandible_mask[row, col, slice_index]}"
            if spinal_cord_mask is not None and 0 <= row < spinal_cord_mask.shape[0] and 0 <= col < spinal_cord_mask.shape[1]:
                info += f", Spinal Cord={spinal_cord_mask[row, col, slice_index]}"
            return info
        ax.format_coord = format_coord
        
        ax.axis('off')
        plt.show()

    # For coronal view, determine slice range along x direction.
    if body_mask is not None:
        start_slice, end_slice = get_nonzero_slice_range(body_mask, slice_dir_indx=0)
    else:
        start_slice = 0
        end_slice = image.shape[0] - 1

    def view_slice_coronal(slice_index):
        fig, ax = plt.subplots(figsize=(10, 10))
        # Display coronal view (slice along x axis).
        ax.imshow(image[slice_index, :, :], cmap='gray', interpolation='none')
        ax.contour(gtv_mask[slice_index, :, :], colors='yellow')
        if body_mask is not None:
            ax.contour(body_mask[slice_index, :, :], colors='orange')
        if mandible_mask is not None:
            ax.contour(mandible_mask[slice_index, :, :], colors='blue')
        if spinal_cord_mask is not None:
            ax.contour(spinal_cord_mask[slice_index, :, :], colors='green')
        
        for idx, coeffs in enumerate(plane_coeffs_list):
            A, B, C = angles_to_vector(coeffs[0], coeffs[1], coeffs[2])
            D = - np.dot([A, B, C], [A, B, C])
            B_val = B if B != 0 else 1e-6
            x = np.linspace(0, image.shape[0], 100)
            z = np.linspace(0, image.shape[2], 100)
            X, Z = np.meshgrid(x, z)
            contour = (-A * X - C * Z - D) / B_val
            ax.contour(Z, X, contour, levels=[slice_index], colors=plane_colors[idx % len(plane_colors)])
        
        # Build legend for coronal view.
        # legend_handles = [Line2D([0], [0], color='yellow', lw=2, label='GTVp')]
        # if body_mask is not None:
        #     legend_handles.append(Line2D([0], [0], color='orange', lw=2, label='Body'))
        # if mandible_mask is not None:
        #     legend_handles.append(Line2D([0], [0], color='blue', lw=2, label='Mandible'))
        # if spinal_cord_mask is not None:
        #     legend_handles.append(Line2D([0], [0], color='green', lw=2, label='Spinal Cord'))
        # for idx, coeffs in enumerate(plane_coeffs_list):
        #     if idx < len(optimization_methods_list):
        #         legend_handles.append(Line2D([0], [0], color=plane_colors[idx % len(plane_colors)], lw=2,
        #                                       label=optimization_methods_list[idx]))
        # ax.legend(handles=legend_handles, loc='upper right')
        
        ax.set_ylim(start_slice, end_slice)
        ax.axis('off')
        plt.show()

    # Interactive slider for axial (z) view.
    slice_slider_axial = widgets.IntSlider(min=0, max=num_slices-1, step=1, value=num_slices//2, description='Axial Slice')
    display(widgets.interact(view_slice_axial, slice_index=slice_slider_axial))
    
    # Interactive slider for coronal (x) view.
    slice_slider_coronal = widgets.IntSlider(min=start_slice, max=end_slice, step=1, value=(start_slice+end_slice)//2, description='Coronal Slice')
    display(widgets.interact(view_slice_coronal, slice_index=slice_slider_coronal))

def display_patient_overlay_structures(struct_dict, title="Overlay Structures"):
    """
    Creates an interactive widget that overlays structure contours over the main image.
    
    The main image is expected under the key "Image" and the primary GTV mask under "GTVp" in struct_dict.
    Other masks (e.g. "Body", "Mandible", "Spinal Cord") are optional. For each slice, the main image is
    displayed in grayscale and the available masks are overlaid as contours using predetermined colors.
    A legend is added for the masks that are present and a custom coordinate formatter shows the pixel 
    coordinates and the mask values when hovering over the image.
    
    Parameters
    ----------
    struct_dict : dict
        Dictionary mapping structure names to (image_array, voxel_size).
        Must contain keys "Image" and "GTVp".
    title : str, optional
        Title prefix for the displayed plot.
    """
    # Check that mandatory keys exist.
    if "Image" not in struct_dict:
        print("Main image (key 'Image') not found in the structure dictionary.")
        return
    if "GTVp" not in struct_dict:
        print("Primary GTV mask (key 'GTVp') not found in the structure dictionary.")
        return
    
    main_img, _ = struct_dict["Image"]
    gtv_img, _  = struct_dict["GTVp"]
    num_slices = main_img.shape[2]
    
    # Define a dictionary of colors for each structure.
    mask_colors = {
        "GTVp": "yellow",   # mandatory
        "Body": "red",
        "Mandible": "blue",
        "Spinal Cord": "green"
    }
    
    # Build list of optional mask keys that are present.
    optional_keys = []
    for key in ["Body", "Mandible", "Spinal Cord"]:
        if key in struct_dict and struct_dict[key] is not None:
            optional_keys.append(key)
    
    def view_slice(slice_index):
        fig, ax = plt.subplots(figsize=(8,8))
        # Display the main image in grayscale.
        ax.imshow(main_img[:, :, slice_index], cmap='gray', interpolation='none')
        
        # Overlay the mandatory GTV mask.
        ax.contour(gtv_img[:, :, slice_index], levels=[0.5], colors=mask_colors["GTVp"], linewidths=2)
        
        # Overlay optional masks if available.
        for key in optional_keys:
            mask_img, _ = struct_dict[key]
            # Only add the contour if there is any nonzero element.
            if np.any(mask_img[:, :, slice_index]):
                ax.contour(mask_img[:, :, slice_index], levels=[0.5], colors=mask_colors[key], linewidths=2)
        
        # Build legend using dummy handles.
        legend_handles = [Line2D([0], [0], color=mask_colors["GTVp"], lw=2, label="GTVp")]
        for key in optional_keys:
            legend_handles.append(Line2D([0], [0], color=mask_colors[key], lw=2, label=key))
        ax.legend(handles=legend_handles, loc='upper right')
        
        # # Set a custom coordinate formatter.
        # def format_coord(x, y):
        #     col = int(round(x))
        #     row = int(round(y))
        #     info = f"x={col}, y={row}, slice={slice_index}"
        #     # Always show value from GTVp.
        #     if 0 <= row < gtv_img.shape[0] and 0 <= col < gtv_img.shape[1]:
        #         info += f", GTVp={gtv_img[row, col, slice_index]}"
        #     # Append info for each optional mask.
        #     for key in optional_keys:
        #         mask_img, _ = struct_dict[key]
        #         if 0 <= row < mask_img.shape[0] and 0 <= col < mask_img.shape[1]:
        #             info += f", {key}={mask_img[row, col, slice_index]}"
        #     return info
        # ax.format_coord = format_coord
        
        ax.set_title(f"{title} - Slice {slice_index}")
        ax.axis('off')
        plt.show()
    
    slider = widgets.IntSlider(min=0, max=num_slices-1, step=1, value=num_slices//2, description='Slice')
    display(widgets.interact(view_slice, slice_index=slider))

def process_all_patient_folders(root_folder, patterns, display_widgets=True):
    """
    Loop over all patient folders in the given root_folder and process each one.
    
    For each patient folder (assumed to be named with a patient ID), the function:
      - Loads the main image and structure mask files using the provided regex patterns,
        transposing each array with np.transpose(array, (1,0,2)).
      - Creates an interactive widget that overlays the structure contours over the main image,
        allowing scrolling through the slices along axis 2.
      - Stores the loaded data in a dictionary keyed by patient ID.
    
    Parameters
    ----------
    root_folder : str
        Directory containing patient folders (each named with a patient ID).
    patterns : dict
        Dictionary mapping structure names (e.g., "image", "gtvp", etc.) to regex patterns.
    display_widgets : bool, optional
        If True, display the interactive widget for each patient (default True).
    
    Returns
    -------
    dict
        A dictionary mapping each patient ID (folder name) to a dictionary with keys:
          - "image": the transposed main image (3D numpy array),
          - "voxel_size": the voxel dimensions,
          - "masks": a dictionary mapping mask names to the transposed 3D mask arrays.
    """
    all_patient_data = {}
    
    # Loop over each item in the root_folder.
    for patient_id in sorted(os.listdir(root_folder)):
        patient_folder = os.path.join(root_folder, patient_id)
        
        if os.path.isdir(patient_folder):
            print(f"Processing patient folder: {patient_id}")
            # Load structures from the patient folder using your patterns.
            struct_data = load_patient_structures(patient_folder, patterns)
            if struct_data:
                all_patient_data[patient_id] = struct_data
                # Display an interactive overlay widget if requested.
                if display_widgets:
                    display_patient_overlay_structures(struct_data, title=f"Patient {patient_id} Overlay")
            else:
                print(f"No matching files found for patient {patient_id}.")
    
    return all_patient_data


## Optimization Pipeline

In [1080]:
def MSP_pipeline(base_path, 
                      output_path,
                      structure_names = ["Image", "GTVp", "Mandible", "Spinal Cord", "Body"],
                      HU_range=[300, 1500],
                      slice_range=None,
                      azimuthal_deg=10,
                      polar_deg=10,
                      uncertainty=90,
                      initialization_steps=10):
                      
                    
    start_pipeline = time.time()
    list_best_plane_params = []
    list_obj_fun = []
    folders = sorted([d for d in os.listdir(base_path)], key=int)
    for folder in folders:
        folder_path = os.path.join(base_path, folder)
        print(f"Processing folder: {folder_path}")
        output_path_patient = os.path.join(output_path, folder)
        os.makedirs(output_path_patient, exist_ok=True)
        struct_dict = load_patient_structures(folder_path, structure_names)
        if struct_dict is None:
            print(f"No matching files found in {folder_path}. Skipping...")
            continue
        else:
            image = struct_dict.get("Image", (None,))[0]
            gtvp = struct_dict.get("GTVp", (None,))[0]
            if image is None or gtvp is None:
                print(f"Image or GTVp not found in {folder_path}. Skipping...")
                continue
            mandibula = struct_dict.get("Mandible", (None,))[0]
            spinalcord = struct_dict.get("Spinal Cord", (None,))[0]
            body = struct_dict.get("Body", (None,))[0]
        
        image = image.astype(np.int16)
        if body is not None:
            image = np.where(body == 1, image, -1000)
        if HU_range is None:
            HU_range = (300, 1500)
        bone_mask = mask_via_threshold(image, HU_range=(HU_range)).astype(np.uint16)
        bone_ct = image * bone_mask
        dental_fillings_mask = mask_via_threshold(image, HU_range=(HU_range[1], 5000)).astype(np.uint16)
        dental_bone_ct = HU_range[1] * dental_fillings_mask
        bone_ct = bone_ct + dental_bone_ct
        image = image * (1 - dental_fillings_mask) + dental_bone_ct
        
        start_slice, end_slice = get_nonzero_slice_range(gtvp)
        if slice_range is not None:
            start_slice = slice_range[0]
            end_slice = slice_range[1] + 1
        
        image = image[:, :, start_slice:end_slice + 1]
        bone_ct = bone_ct[:, :, start_slice:end_slice + 1]
        gtvp = gtvp[:, :, start_slice:end_slice + 1]
        # For optional structures, only apply slicing if they exist.
        if body is not None:
            body = body[:, :, start_slice:end_slice + 1]
        if mandibula is not None:
            mandibula = mandibula[:, :, start_slice:end_slice + 1]
        if spinalcord is not None:
            spinalcord = spinalcord[:, :, start_slice:end_slice + 1]
        
        interpolator_path = os.path.join(output_path_patient, 'interpolator.joblib')
        shape = image.shape
        grid_y = np.arange(shape[0])
        grid_x = np.arange(shape[1])
        grid_z = np.arange(shape[2])

        if os.path.exists(interpolator_path):
            interpolator = joblib.load(interpolator_path)
        else:
            start_interpolator = time.time()
            interpolator = RegularGridInterpolator((grid_y, grid_x, grid_z), image, 
                                                   method='cubic', bounds_error=False, fill_value=None)
            end_interpolator = time.time()
            print(f"Cubic interpolator took {end_interpolator - start_interpolator:.2f} seconds.")
            joblib.dump(interpolator, interpolator_path)

        # mse1 = compute_objective(np.array([np.deg2rad(180), np.deg2rad(62), 225]), image, interpolator)
        # plot_middle_slice_with_planes(image, [np.array([np.deg2rad(180), np.deg2rad(62), -225])], title='Middle Slice with Initial Plane')
        # mse2 = compute_objective(np.array([np.deg2rad(90), np.deg2rad(90), 255]), image, interpolator)
        # plot_middle_slice_with_planes(image, [np.array([np.deg2rad(90), np.deg2rad(90), 255])], title='Middle Slice with Initial Plane')
        #theta, phi, L = np.pi / 2, np.pi / 2, 250

        #plot_middle_slice_with_planes(image, [np.array([np.pi / 4, np.pi / 2, 300])], title='Middle Slice with Initial Plane')
        #plot_mse_vs_parameters(bone_ct, interpolator, np.pi/2, np.pi/2, 250, output_path_patient, "1")
        
        # Initialize candidate plane parameters using a grid search.
        initial_plane, initial_mse = parameter_initialization(image, bone_ct, output_path_patient, interpolator,
                                                    azimuthal_deg=azimuthal_deg, polar_deg=polar_deg, uncertainty=uncertainty,
                                                    initialization_steps=initialization_steps)
        initial_plane_deg = np.rad2deg(initial_plane[0]), np.rad2deg(initial_plane[1]), initial_plane[2]
        print(f"Initial plane parameters: {initial_plane_deg} with MSE {round(initial_mse, 2)}")
        plot_middle_slice_with_planes(image, [initial_plane], title='Middle Slice with Initial Plane', output_path=output_path_patient, filename=f'Image_slice_with_initial_plane.svg')
        break
        if os.path.exists(os.path.join(output_path_patient, "params_array.npy")): 
            plane_params = np.load(os.path.join(output_path_patient, "params_array.npy"))
            best_plane_params = plane_params[-1]
            #distances, normal_vector, indices_coord, indices_im = compute_signed_distances(best_plane_params, gtvp)
            display_scrollable_slices_with_plane(image, gtvp, body, mandibula, spinalcord, [best_plane_params], [optimization_method])
            
            # second_path = os.path.join(r"/home/loriskeller/Documents/Master Project/VS/Data_extract_and_midline/Results/results_07_04_25/Midsagplanes_HU300to2500", f'{pat_id}')
            # second_plane_params = np.load(os.path.join(second_path, "params_array.npy"))
            # second_best_plane_params = second_plane_params[-1]
            # display_scrollable_slices_with_plane(image, gtvp, body, mandibula, spinalcord, [best_plane_params,second_best_plane_params], [f"900-2000", f"300-2000"])
        else:
            # Optimize the plane parameters using the BFGS method.
            start_optimization = time.time()
            opt_result = optimize_plane(initial_plane, bone_ct, interpolator, interpolators_gradient)
            end_optimization = time.time()
            best_plane_params = opt_result.x  # Optimized [theta, phi, L].
            obj_fun = opt_result.fun
            print(f"Optimization ({optimization_method}) took {end_optimization - start_optimization:.2f} seconds.")
            print(f"Optimized parameters: {np.rad2deg(best_plane_params[0])}, {np.rad2deg(best_plane_params[1])}, {best_plane_params[2]} with MSE {round(obj_fun, 2)}")
            best_plane_result = {'params': best_plane_params, 'obj_fun': obj_fun}
            # Assuming res is the optimization result from optimize_plane.
            params_array = np.array(opt_result.params_list)
            objective_array = np.array(opt_result.objective_value_list)
            # Save as .npy files.
            np.save(os.path.join(output_path_patient, "params_array.npy"), params_array)
            np.save(os.path.join(output_path_patient, "objective_array.npy"), objective_array)

            if patient is None:
                list_best_plane_params.append(best_plane_params)
                list_obj_fun.append(obj_fun)
        
            # Plot the middle slice with the optimized plane.
            #plot_middle_slice_with_planes(image, [best_plane_params], title='Middle Slice with Optimized Plane', output_path=output_path_patient, filename=f'Image_slice_with_plane.svg')
            #display_scrollable_slices_with_plane(image, gtvp, body, mandibula, spinalcord, [best_plane_params], [optimization_method])
        
        # Optionally, display additional results.
        # if results_path_list is not None:
        #     results_list = [np.load(path) for path in results_path_list]
        #     display_scrollable_slices_with_plane(image, body, gtvp, mandibula, spinalcord, results_list, optimization_methods_list)
        
        
    # Suppose list_best_plane_params and list_obj_fun are Python lists containing numeric values or arrays.
    # if not os.path.exists(output_path):
    #     # Save the results as a compressed .npz file.
    #     np.savez_compressed(os.path.join(output_path, "best_params_and_objectives.npz"),
    #                 best_plane_params=list_best_plane_params,
    #                 objective_values=list_obj_fun)
        

    end_pipeline = time.time()
    print(f"Total time for processing all patients: {end_pipeline - start_pipeline:.2f} seconds")

## Mapping to real space

In [1081]:
def physical_plane_params(theta, phi, L, voxel_size):
    """
    Convert plane parameters from voxel space to physical space.

    Parameters:
        theta, phi, L : float
                Plane parameters in voxel space.
        voxel_size : np.array
                Voxel size.

    Returns:
        params_real : np.array
                Plane parameters in physical space.
    """
    
    vec_normalized = np.array([np.cos(phi)*np.cos(theta),
                    np.cos(phi)*np.sin(theta),
                    np.sin(phi)])
    vec = L * vec_normalized
    vec_real = np.array([vec[0] * voxel_size[0],
                         vec[1] * voxel_size[1],
                         vec[2] * voxel_size[2]])
    theta_real, phi_real, L_real = vector_to_angles(vec_real)
    params_real = np.array([theta_real, phi_real, L_real])

    return params_real

def real_distance_to_plane(x_voxel, theta, phi, L, voxel_size):
    """
    Compute the physical distance (mm) from a set of voxel points to the plane defined by theta, phi, and L.
    
    Parameters:
      x_voxel : np.array of shape (n,3)
          The array of voxel coordinates, where each row is [x, y, z].
      theta, phi, L : float
          Plane parameters in voxel space.
      voxel_size : array-like, shape (3,)
          Voxel dimensions as [s_x, s_y, s_z].
    
    Returns:
      d : np.array
          A 1D array of signed distances (in mm) for each voxel.
    """
    # Convert voxel coordinates to physical coordinates via broadcasting.
    x_phys = x_voxel * np.array(voxel_size)  # Each column multiplied by corresponding voxel size.
    
    # Convert plane parameters to physical space.
    theta_real, phi_real, L_real = physical_plane_params(theta, phi, L, voxel_size)
    
    # Compute signed distance for each voxel.
    d = (np.cos(phi_real)*np.cos(theta_real)*x_phys[:, 0] +
         np.cos(phi_real)*np.sin(theta_real)*x_phys[:, 1] +
         np.sin(phi_real)*x_phys[:, 2] - L_real)
    
    return d


## Verification Plots

In [1082]:
def plot_mse_vs_parameters(image, interpolator_intensity,
                           theta_best, phi_best, L_best, output_path, pat):
    """
    Generate plots of the mean squared error (MSE) as a function of θ, φ, and L.
    
    For each parameter, a range is defined around the best value and the objective
    function is evaluated while holding the other two parameters fixed.
    
    Parameters
    ----------
    image : numpy.ndarray
        The 3D image.
    interpolator_intensity : RegularGridInterpolator
        Interpolator for the image intensity.
    interpolators_gradient : dict
        Dictionary with keys 'x', 'y', and 'z' containing RegularGridInterpolator 
        objects for the image gradients.
    theta_best : float
        Best theta value (in radians).
    phi_best : float
        Best phi value (in radians).
    L_best : float
        Best L value.
    output_path : str
        Directory where the plots will be saved.
    pat : str or int
        Identifier (e.g. patient number) used in filenames.
    """
    # Define parameter ranges around the best values.
    # For theta and phi, use ±2 degrees (converted to radians)
    delta_rad = np.deg2rad(2)
    theta_range = np.linspace(theta_best - delta_rad, theta_best + delta_rad, 10)
    phi_range   = np.linspace(phi_best - delta_rad, phi_best + delta_rad, 10)
    # For L, use ±4 units around L_best.
    L_range     = np.linspace(L_best - 4, L_best + 4, 10)
    
    mse_theta = []
    mse_phi   = []
    mse_L     = []
    
    # Compute MSE vs. Theta (with φ and L fixed)
    for theta in theta_range:
        mse = compute_objective(np.array([theta, phi_best, L_best]),
                                image, interpolator_intensity)
        mse_theta.append(mse)
    
    # Compute MSE vs. Phi (with θ and L fixed)
    for phi in phi_range:
        mse = compute_objective(np.array([theta_best, phi, L_best]),
                                image, interpolator_intensity)
        mse_phi.append(mse)
    
    # Compute MSE vs. L (with θ and φ fixed)
    for L in L_range:
        mse = compute_objective(np.array([theta_best, phi_best, L]),
                                image, interpolator_intensity)
        mse_L.append(mse)
    
    # Plot MSE vs. Theta
    plt.figure()
    plt.plot(np.degrees(theta_range), mse_theta, label='MSE vs. Theta')
    plt.xlabel('Theta (degrees)')
    plt.ylabel('MSE')
    plt.title('MSE vs. Theta\nBest Theta = {:.2f}°'.format(np.degrees(theta_best)))
    plt.legend()
    plt.grid(True)
    plt.show()
    theta_filename = os.path.join(output_path, f'mse_vs_theta_{pat}.svg')
    #plt.savefig(theta_filename)
    
    # Plot MSE vs. Phi
    plt.figure()
    plt.plot(np.degrees(phi_range), mse_phi, label='MSE vs. Phi')
    plt.xlabel('Phi (degrees)')
    plt.ylabel('MSE')
    plt.title('MSE vs. Phi\nBest Phi = {:.2f}°'.format(np.degrees(phi_best)))
    plt.legend()
    plt.grid(True)
    plt.show()
    phi_filename = os.path.join(output_path, f'mse_vs_phi_{pat}.svg')
    #plt.savefig(phi_filename)
    
    # Plot MSE vs. L
    plt.figure()
    plt.plot(L_range, mse_L, label='MSE vs. L')
    plt.xlabel('L')
    plt.ylabel('MSE')
    plt.title('MSE vs. L\nBest L = {:.2f}'.format(L_best))
    plt.legend()
    plt.grid(True)
    plt.show()
    L_filename = os.path.join(output_path, f'mse_vs_L_{pat}.svg')
    #plt.savefig(L_filename)
    
    plt.show()
    print("MSE plots saved successfully:")
    print(f"  Theta plot: {theta_filename}")
    print(f"  Phi plot:   {phi_filename}")
    print(f"  L plot:     {L_filename}")


## Output

In [1083]:
#%matplotlib widget
%matplotlib inline

data_path = r"/home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/Patient_structures_clean"
output_path = r"/home/loriskeller/Documents/Master Project/Results/12.05.25"

MSP_pipeline(data_path, output_path, structure_names=["Image", "GTVp", "Mandible", "Spinal Cord", "Body"],
                      HU_range=[300, 1500], slice_range=None, azimuthal_deg=90, polar_deg=90, uncertainty = 90, initialization_steps=20)

Processing folder: /home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/Patient_structures_clean/1
Loaded 'Body' from /home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/Patient_structures_clean/1/Body.nii.gz
Loaded 'Mandible' from /home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/Patient_structures_clean/1/Mandible.nii.gz
Loaded 'Image' from /home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/Patient_structures_clean/1/image.nii.gz
Loaded 'Spinal Cord' from /home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/Patient_structures_clean/1/Spinal Cord.nii.gz
Loaded 'GTVp' from /home/loriskeller/Documents/Master Project/Patient data/patient_data_complete/Patient_structures_clean/1/GTVp.nii.gz
Starting parameter initialization...
Azimuthal angles: [  0.           9.47368421  18.94736842  28.42105263  37.89473684
  47.36842105  56.84210526  66.31578947  75.7894736

IndexError: index 380 is out of bounds for axis 0 with size 3